In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

data = []
base_url = "https://www.betman.co.kr/main/mainPage/gamebuy/closedGameSlipIFR.do"
params_template = {
    "gmId": "G101",
    "gameDivCd": "C",
    "isIFR": "Y"
}

# 연도별 회차 반복 (2015년 ~ 2024년, 1~157회차)
for year in tqdm(range(15, 25), desc="연도"):
    for round_num in range(1, 158):
        gm_ts = f"{year:02}{round_num:03}"
        params = params_template.copy()
        params["gmTs"] = gm_ts

        try:
            response = requests.get(base_url, params=params, timeout=5)
            soup = BeautifulSoup(response.content, "html.parser")
            rows = soup.select("tbody#tbd_gmBuySlipList > tr")
            if not rows:
                continue  # 페이지 없음

            for row in rows:
                try:
                    # 종목: td[3], 게임유형: td[4]
                    league = row.select_one("td:nth-of-type(3)").get_text(strip=True)
                    gametype = row.select_one("td:nth-of-type(4)").get_text(strip=True)
                    if "KBO" not in league or "일반" not in gametype:
                        continue  # 야구/일반 베팅이 아니면 제외

                    matchup = row.select_one("td:nth-of-type(5)").get_text(" ", strip=True)
                    scores = row.select("td:nth-of-type(5) strong.score")
                    home_score = scores[0].get_text(strip=True) if len(scores) > 0 else ""
                    away_score = scores[1].get_text(strip=True) if len(scores) > 1 else ""

                    odds_btns = row.select("td:nth-of-type(6) button span.db")
                    odds = [btn.get_text(strip=True).replace("배당률 상승", "").replace("배당률 하락", "") for btn in odds_btns]
                    
                    date = row.select_one("td:nth-of-type(7)").get_text(" ", strip=True)

                    data.append({
                        "연도": 2000 + year,
                        "회차": round_num,
                        "날짜": date,
                        "경기": matchup,
                        "홈팀 점수": home_score,
                        "원정팀 점수": away_score,
                        "배당1": odds[0] if len(odds) > 0 else "",
                        "배당2": odds[1] if len(odds) > 1 else "",
                        "배당3": odds[2] if len(odds) > 2 else ""
                    })
                except:
                    continue
            time.sleep(0.5)
        except:
            continue

# 저장
df = pd.DataFrame(data)
df.to_excel("KBO_betman_cleaned.xlsx", index=False)


연도:   0%|          | 0/10 [00:00<?, ?it/s]